In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from os.path import isdir, join
import librosa
from tqdm import tqdm
# Any results you write to the current directory are saved as output.
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

['train', 'sample_submission.csv']
Default GPU Device: /device:GPU:0


In [2]:
# os.listdir("../input/dataasdfasd/")
# audio_file_path = "../input/dataasdfasd/output.wav"
# os.linesep()
# samps,sr = librosa.load(audio_file_path, mono=True, sr=None)
# mfcc = librosa.feature.mfcc(samps, sr = sr)z
# pad_width = max_pad - mfcc.shape[1]
# mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
audio_path = "../input/train/audio/"

In [3]:
# all audio dirs
dirs = [f for f in os.listdir(audio_path) if os.path.isdir(join(audio_path, f))]
dirs.sort()

In [4]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(mfcc, x_axis='time')
# plt.colorbar()
# plt.title('MFCC')
# plt.tight_layout()

In [5]:
import librosa.display

In [6]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(mfcc, x_axis='time')
# plt.colorbar()
# plt.title('MFCC')
# plt.tight_layout()

In [7]:
import os
import librosa
import numpy as np
class AudioFeatureDataset():
    def __init__(self,file_path):
        self.file_path = file_path
        self.labels = os.listdir(self.file_path)
        self.target_labels = ['no', 'left', 'nine', 'two', 'yes', 'bed', 'stop', 'cat', 'dog', 'off']
        self.data_dict = {}
        for tl in self.target_labels:
            files_dir = os.path.join(file_path,tl)
            all_audio_fp_s =[ os.path.join(files_dir,f) for f in os.listdir(files_dir)]
            self.data_dict[tl] = all_audio_fp_s
    
    def process(self,file,max_pad = 32):
        samps,sr = librosa.load(file, mono=True, sr=None)
#         mfcc = librosa.feature.mfcc(samps, sr = sr)
#         pad_width = max_pad - mfcc.shape[1]
#         mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        pad_len = 16000 - samps.shape[0]
        samps= np.pad(samps,(0,pad_len),'constant')
        return np.array(samps)
    def get_dataset(self):
        labels =  []
        features = []
        for t in tqdm(self.target_labels):
            for fp in self.data_dict[t]:
                labels.append(t)
                features.append(self.process(fp))
        return labels, features

In [8]:
a = AudioFeatureDataset(audio_path)

In [9]:
Y,x = a.get_dataset()

100%|██████████| 10/10 [00:25<00:00,  2.50s/it]


In [10]:
x = np.array(x)
Y = np.array(Y)

In [11]:
for i in range(len(x)):
    pad_len = 16000 - x[i].shape[0]
    x[i] = np.pad(x[i],(0,pad_len),'constant')

In [12]:
x = x.reshape(21771, 1, 16000) 

In [13]:
z  = [[1,2],[1,2]]
z = np.array(z)

In [14]:
x.shape

(21771, 1, 16000)

In [15]:
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio=0.6, random_state=42):
    return train_test_split(x, Y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [16]:
X_train, X_test, y_train, y_test = get_train_test()

In [17]:
# X_train[0].shape

In [18]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(X_train[3], x_axis='time')
# plt.colorbar()
# plt.title('MFCC')
# plt.tight_layout()

In [19]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [20]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [21]:
labelencoder.fit_transform(y_test)

array([9, 4, 8, ..., 3, 9, 5])

In [22]:
mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))

In [23]:
mapping

{'bed': 0,
 'cat': 1,
 'dog': 2,
 'left': 3,
 'nine': 4,
 'no': 5,
 'off': 6,
 'stop': 7,
 'two': 8,
 'yes': 9}

In [24]:
y_test = labelencoder.fit_transform(y_test)
mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))

In [25]:
mapping

{'bed': 0,
 'cat': 1,
 'dog': 2,
 'left': 3,
 'nine': 4,
 'no': 5,
 'off': 6,
 'stop': 7,
 'two': 8,
 'yes': 9}

In [26]:
y_test

array([9, 4, 8, ..., 3, 9, 5])

In [27]:
y_train = labelencoder.fit_transform(y_train)
mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))

In [28]:
!pip install kapre

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [29]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [30]:
import keras
import kapre
from keras.models import Sequential
from keras.layers import Dense,AveragePooling2D
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise

# 6 channels (!), maybe 1-sec audio signal, for an example.
input_shape = (1,16000)
sr = 16000
model = Sequential()
# A mel-spectrogram layer
model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                         padding='same', sr=sr, n_mels=128,
                         fmin=0.0, fmax=sr/2, power_melgram=1.0,
                         return_decibel_melgram=True,trainable_fb=False,
                         trainable_kernel=False,
                         name='trainable_stft'))
# Maybe some additive white noise.
model.add(AdditiveNoise(power=0.1))
# If you wanna normalise it per-frequency
model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
# After this, it's just a usual keras workflow. For example..
# Add some layers, e.g., model.add(some convolution layers..)
# Compile the model
model.add(Conv2D(39, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(20, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(15, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(25, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(22, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(22, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(25, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(39, kernel_size=(3, 3), activation='relu',dim_ordering="th"))
model.add(Conv2D(45, kernel_size=(3, 3), activation='relu',dim_ordering="th"))

model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(12))
model.add(Dense(10, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.adam(),metrics=['accuracy'])
model.summary() 

Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(20, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(39, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(15, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(39, kernel_size=(3, 3), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, kernel_size=(3, 3), activation="relu", d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
trainable_stft (Melspectrogr (None, 128, 32, 1)        296064    
_________________________________________________________________
additive_noise_1 (AdditiveNo (None, 128, 32, 1)        0         
_________________________________________________________________
normalization2d_1 (Normaliza (None, 128, 32, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 126, 30, 39)       390       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 28, 37)        22700     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 39, 26, 35)        7059      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 24, 33)        5280      
__________

In [31]:
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [32]:
# X_train.reshape(-1,13062,16000).shape

In [33]:

# X_test = X_test.reshape(X_test.shape[0], 20, 32, 1)

In [34]:
# a = X_train[0].reshape(1,16000)

In [35]:
# model.predict(np.array([a]))

In [36]:
from keras.optimizers import Optimizer
from keras import backend as K
import six
import copy
from six.moves import zip
from keras.utils.generic_utils import serialize_keras_object
from keras.utils.generic_utils import deserialize_keras_object
from keras.legacy import interfaces
class AdamW(Optimizer):
    """Adam optimizer.
    Default parameters follow those provided in the original paper.
    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Decoupled weight decay over each update.
    # References
        - [Adam - A Method for Stochastic Optimization](http://arxiv.org/abs/1412.6980v8)
        - [Optimization for Deep Learning Highlights in 2017](http://ruder.io/deep-learning-optimization-2017/index.html)
        - [Fixing Weight Decay Regularization in Adam](https://arxiv.org/abs/1711.05101)
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/6)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.init_lr = lr # decoupled weight decay (2/6)
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.wd = K.variable(weight_decay, name='weight_decay') # decoupled weight decay (3/6)
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd # decoupled weight decay (4/6)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))
        eta_t = lr / self.init_lr # decoupled weight decay (5/6)

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - eta_t * wd * p # decoupled weight decay (6/6)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [37]:
# model = Sequential()
# model.add(Conv2D(64, kernel_size=(20, 8), activation='relu', input_shape=(20, 32, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
# model.add(Dropout(0.25))
# model.add(Conv2D(64, kernel_size=(10, 4), activation='relu',dim_ordering="th"))
# model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
# model.add(Dense(10, activation='softmax'))
# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.adam(),metrics=['accuracy'])

In [38]:
model.fit(X_train[1000:], y_train[1000:], batch_size=100, epochs=100, verbose=1, validation_data=(X_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 12062 samples, validate on 8709 samples
Epoch 1/100
12062/12062 [==============================] - 7s 546us/step - loss: 2.1498 - acc: 0.1954 - val_loss: 1.5195 - val_acc: 0.4351
Epoch 2/100
12062/12062 [==============================] - 4s 307us/step - loss: 1.0175 - acc: 0.6448 - val_loss: 0.7041 - val_acc: 0.7637
Epoch 3/100
12062/12062 [==============================] - 4s 309us/step - loss: 0.6509 - acc: 0.7861 - val_loss: 0.5293 - val_acc: 0.8266
Epoch 4/100
12062/12062 [==============================] - 4s 308us/step - loss: 0.4397 - acc: 0.8612 - val_loss: 0.4016 - val_acc: 0.8745
Epoch 5/100
12062/12062 [==============================] - 4s 310us/step - loss: 0.3301 - acc: 0.8983 - val_loss: 0.3361 - val_acc: 0.8916
Epoch 6/100
12062/12062 [==============================] - 4s 310us/step - loss: 0.2717 - acc: 0.9166 - val_loss: 0.2754 - val_acc: 0.9119
Epoch 7/100
12062/12062 [==============================] - 4s 308us/s

In [39]:
print(model.output.op.name)
print(model.input.op.name)

dense_2/Softmax
trainable_stft_input


In [40]:
%%time
count = 0
for i in range(1000):    
    tesval = np.array([X_train[i]]).reshape(1,1,16000)
    if np.argmax(model.predict(tesval)) ==  np.argmax(y_train[i]):
        count+= 1
print(count)

872
CPU times: user 2.72 s, sys: 148 ms, total: 2.87 s
Wall time: 2.17 s


In [41]:
np.argmax(model.predict(tesval))

1

In [42]:
model.output.op.name

'dense_2/Softmax'

In [43]:
model.save_weights('94_cnn.h5',keras)

In [44]:
model.save("yo.h5")
import tensorflow as tf

In [45]:
from tensorflow.contrib import lite
converter = lite.TFLiteConverter.from_keras_model_file( 'yo.h5')
tfmodel = converter.convert()
converter.allow_custom_ops=True
open ("model.tflite" , "wb") .write(tfmodel)


ValueError: Unknown layer: Melspectrogram